All rows except the pivot row can be divided by the previous pivot element.  
$$
initial\ tableau:
\begin{bmatrix}
a_1 & a_2& a_3 \\
b_1 & b_2 & b_3 \\
c_1 & c_2 & c_3
\end{bmatrix}
$$
choose the initial pivot as $a_2$
$$
\Rightarrow
\begin{bmatrix}
a_1 & a_2& a_3 \\
a_2b_1-a_1b_2 & 0 & a_2b_3-a_3b_2 \\
a_2c_1-a_1c_2 & 0 & a_2c_3-a_3c_2
\end{bmatrix}
$$
without losing generality, choose $a_2b_1-a_1b_2$ as next pivot:
$$
\Rightarrow
\begin{bmatrix}
0 & a_2(a_2b_1-a_1b_2)& a_3(a_2b_1-a_1b_2)-a_1(a_2b_3-a_3b_2) \\
a_2b_1-a_1b_2 & 0 & a_2b_3-a_3b_2 \\
0 & 0 & (a_2c_3-a_3c_2)(a_2b_1-a_1b_2)-(a_2c_1-a_1c_2)(a_2b_3-a_3b_2)
\end{bmatrix}
$$
$$
\Rightarrow
\begin{bmatrix}
0 & a_2(a_2b_1-a_1b_2)& a_2(a_3b_1-a_1b_3) \\
a_2b_1-a_1b_2 & 0 & a_2b_3-a_3b_2 \\
0 & 0 & a_2(some\ constant)
\end{bmatrix}
$$
The entry $a_2(some\ constant)$ is got because the only term $a_3c_2a_1b_2$ that has no $a_2$ is cancalled by $-a_3c_2a_1b_2$  
Proof done.

In [4]:
import numpy as np

In [5]:
A = np.array([[3, 3], 
              [2, 5], 
              [0, 6]])
B_T = np.array([[3, 2, 3], 
                [2, 6, 1]])

In [46]:
def pivoting(tableau, basic_vars, pivot, previous_pivot):
    with np.errstate(divide='ignore'):
        ratios = tableau[:, -1] / tableau[:, pivot]
    row_min = ratios.argmin()
    element = tableau[row_min, pivot]
    for i in range(tableau.shape[0]):
        if i != row_min:
            tableau[i, :] -= tableau[i, pivot] * tableau[row_min, :] / element
            tableau[i, :] *= element
            tableau[i, :] /= previous_pivot
    basic_vars[row_min], pivot = pivot, basic_vars[row_min]
    return pivot, element

In [47]:
def create_tableau(A, B):
    B_T = B.T
    m, n = A.shape
    tableaus = []
    for i in range(2):
        tableaus.append(np.empty((A.shape[1-i], m+n+1)))
        tableaus[i][:, :m] = [B_T, np.identity(m)][i]
        tableaus[i][:, m:m+n] = [np.identity(n), A][i]
        tableaus[i][:, -1] = 1
    basic_vars_list = [np.arange(m, m+n), np.arange(m)]
    return tableaus, basic_vars_list

In [71]:
def Lemke_Howson(A, B, init_pivot):
    m, n = A.shape
    pivot = init_pivot
    tableaus, basic_vars_list = create_tableau(A, B)
    players = [init_pivot // m, 1 - init_pivot // m]
    #store the pivots of previous turn
    pivots_element = [1, 1]
    while True:
        for i in players:
            pivot, pivots_element[i] = pivoting(tableaus[i], basic_vars_list[i], pivot, pivots_element[i])
            print(tableaus[i], basic_vars_list[i])
            if pivot == init_pivot:
                break
        else:
            continue
        break
    #summarize the found NE
    out = np.zeros(m+n)
    for i, (start, num) in enumerate(zip([0, m], [m, n])):
        ind = basic_vars_list[i] < start + num if i == 0 else start <= basic_vars_list[i]
        out[basic_vars_list[i][ind]] = tableaus[i][ind, -1]
        out[start:start+num] /= out[start:start+num].sum()
    
    print(out[:m], out[m:])

In [72]:
Lemke_Howson(A, B_T.T, 1)

[[ 14.   0.  16.   6.  -2.   4.]
 [  2.   6.   1.   0.   1.   1.]] [3 1]
[[  6.   0.  -3.  18.   0.   3.]
 [  0.   6.  -5.  12.   0.   1.]
 [  0.   0.   1.   0.   6.   1.]] [0 1 4]
[[ 14.   0.  16.   6.  -2.   4.]
 [  3.  16.   0.  -1.   3.   2.]] [2 1]
[[ 12. -18.   9.   0.   0.   3.]
 [  0.   6.  -5.  12.   0.   1.]
 [  0.   0.   2.   0.  12.   2.]] [0 3 4]
[ 0.          0.33333333  0.66666667] [ 0.33333333  0.66666667]
